In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Adrenocortical_Cancer"
cohort = "GSE68606"

# Input paths
in_trait_dir = "../../input/GEO/Adrenocortical_Cancer"
in_cohort_dir = "../../input/GEO/Adrenocortical_Cancer/GSE68606"

# Output paths
out_data_file = "../../output/preprocess/Adrenocortical_Cancer/GSE68606.csv"
out_gene_data_file = "../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE68606.csv"
out_clinical_data_file = "../../output/preprocess/Adrenocortical_Cancer/clinical_data/GSE68606.csv"
json_path = "../../output/preprocess/Adrenocortical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"caArray_dobbi-00100: Interlaboratory comparability study of cancer gene expression analysis using oligonucleotide microarrays"
!Series_summary	"A key step in bringing gene expression data into clinical practice is the conduct of large studies to confirm preliminary models. The performance of such confirmatory studies and the transition to clinical practice requires that microarray data from different laboratories are comparable and reproducible. We designed a study to assess the comparability of data from four laboratories that will conduct a larger microarray profiling confirmation project in lung adenocarcinomas. To test the feasibility of combining data across laboratories, frozen tumor tissues, cell line pellets, and purified RNA samples were analyzed at each of the four laboratories. Samples of each type and several subsamples from each tumor and each cell line were blinded before being distributed. The laboratories followed a common protocol

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Step 1: Determine gene expression data availability
# The background information mentions "gene expression analysis using oligonucleotide microarrays"
# and "Affymetrix Human Genome U133A arrays", which indicates gene expression data
is_gene_available = True

# Step 2: Determine variable availability and create conversion functions
# 2.1 Data Availability

# For trait (Adrenocortical Cancer), we need to check disease state or histology
# From sample characteristics, we can see "disease state: Adrenal Cortical Adenoma" in key 1
# and "histology: Adrenal Cortical Adenoma" in key 7
# Let's use disease state (key 1) as it's more standardized
trait_row = 1

# Age information is available in key 6
age_row = 6

# Gender (Sex) information is available in key 5
gender_row = 5

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait values to binary (0=no disease, 1=has disease)"""
    if value is None or "--" in value:
        return None
    
    # Extract value after colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # For Adrenocortical_Cancer, we check for adrenal-related conditions
    if "Adrenal Cortical Adenoma" in value:
        return 1  # Has adrenocortical condition
    else:
        return 0  # Does not have adrenocortical condition

def convert_age(value):
    """Convert age values to continuous numerical values"""
    if value is None or "--" in value:
        return None
    
    # Extract value after colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender values to binary (0=female, 1=male)"""
    if value is None or "--" in value:
        return None
    
    # Extract value after colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value == "female":
        return 0
    elif value == "male":
        return 1
    else:
        return None

# Step 3: Save metadata about dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction
# If trait data is available, extract clinical features
if trait_row is not None:
    # Assuming clinical_data is available from a previous step
    selected_clinical_df = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM1676864': [nan, nan, nan], 'GSM1676865': [nan, nan, nan], 'GSM1676866': [0.0, nan, 0.0], 'GSM1676867': [nan, nan, nan], 'GSM1676868': [nan, nan, nan], 'GSM1676869': [nan, nan, nan], 'GSM1676870': [nan, nan, nan], 'GSM1676871': [nan, nan, nan], 'GSM1676872': [nan, nan, nan], 'GSM1676873': [nan, nan, nan], 'GSM1676874': [0.0, 67.0, 1.0], 'GSM1676875': [0.0, 66.0, 1.0], 'GSM1676876': [0.0, 72.0, 1.0], 'GSM1676877': [0.0, 56.0, 0.0], 'GSM1676878': [0.0, 48.0, 0.0], 'GSM1676879': [nan, nan, nan], 'GSM1676880': [nan, nan, nan], 'GSM1676881': [0.0, nan, nan], 'GSM1676882': [0.0, nan, nan], 'GSM1676883': [0.0, nan, nan], 'GSM1676884': [0.0, nan, nan], 'GSM1676885': [0.0, nan, nan], 'GSM1676886': [0.0, nan, nan], 'GSM1676887': [0.0, nan, nan], 'GSM1676888': [nan, nan, nan], 'GSM1676889': [1.0, nan, nan], 'GSM1676890': [0.0, nan, nan], 'GSM1676891': [0.0, nan, nan], 'GSM1676892': [0.0, nan, nan], 'GSM1676893': [0.0, nan, nan], 'GSM1676894': [0.0, nan, 

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the identifiers shown, these appear to be Affymetrix probe IDs (e.g., "1007_s_at", "1053_at")
# rather than standard human gene symbols (like "TP53", "BRCA1", etc.)
# These probe IDs need to be mapped to human gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe gene identifiers and annotation data
# In gene_data, identifiers are like '1007_s_at', '1053_at', etc. (indexed)
# In gene_annotation, 'ID' column has the same format: '1007_s_at', '1053_at', etc.
# The 'Gene Symbol' column contains the human gene symbols we need: 'DDR1 /// MIR4640', 'RFC2', etc.

# 2. Get gene mapping dataframe by extracting ID and Gene Symbol columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# 4. Preview the mapped gene expression data
print("First 10 genes after mapping:")
print(gene_data.index[:10])

# 5. Preview a small portion of the data to verify the mapping worked correctly
print("\nPreview of gene expression data:")
print(preview_df(gene_data.iloc[:5, :5]))

# 6. Save the gene expression data to the specified output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


First 10 genes after mapping:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB',
       'AAK1', 'AAMDC'],
      dtype='object', name='Gene')

Preview of gene expression data:
{'GSM1676864': [411.658, 9.93369, 67.9692, 103.083, 253.048], 'GSM1676865': [328.766, 29.7297, 42.2613, 19.8402, 307.313], 'GSM1676866': [283.038, 31.5178, 103.195, 120.743, 301.599], 'GSM1676867': [441.273, 117.207, 34.8804, 233.94, 54.3385], 'GSM1676868': [456.17, 66.0545, 37.4472, 132.972, 292.66]}


Gene expression data saved to ../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE68606.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols using NCBI Gene database...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization, gene data shape: {gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Load previously saved clinical data
clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
linked_data = geo_link_clinical_genetic_data(clinical_data, gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data systematically
linked_data = handle_missing_values(linked_data, trait)
print(f"After handling missing values, linked data shape: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_biased}")

# 5. Conduct final quality validation and save cohort information
note = "Dataset containing gene expression profiles of various cancer types including adrenocortical tumors."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies.")

Normalizing gene symbols using NCBI Gene database...
After normalization, gene data shape: (13542, 137)


Normalized gene expression data saved to ../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE68606.csv
Linked data shape: (137, 13545)


After handling missing values, linked data shape: (88, 13545)
For the feature 'Adrenocortical_Cancer', the least common label is '1.0' with 6 occurrences. This represents 6.82% of the dataset.
The distribution of the feature 'Adrenocortical_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 61.8
  50% (Median): 61.8
  75%: 61.8
Min: 48.0
Max: 72.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 9 occurrences. This represents 10.23% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Is trait biased: False


Linked data saved to ../../output/preprocess/Adrenocortical_Cancer/GSE68606.csv
